In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate
import json
%config Completer.use_jedi = False  # Fixes autocomplete issues
%config InlineBackend.figure_format = 'retina'  # Improves plot resolution

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp

In [ ]:
common_resampled_rate = 1000 #in Hz
event_name = "Apply halt: 2s" #event to align data to, can make into list in the future?

#PATH TO Cohort 1 visual mismatch THIS
rawdata_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03')
data_path = rawdata_path.parent / f"{rawdata_path.name}_processedData/downsampled_data"
save_path = rawdata_path.parent / f"{rawdata_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])


photometry_tracking_encoder_data = pd.read_parquet(data_path / "photometry_tracking_encoder_data.parquet", engine="pyarrow")
camera_photodiode_data = pd.read_parquet(data_path / "camera_photodiode_data.parquet", engine="pyarrow")
experiment_events = pd.read_parquet(data_path / "experiment_events.parquet", engine="pyarrow")
photometry_info = pd.read_parquet(data_path / "photometry_info.parquet", engine="pyarrow")
session_settings = pd.read_parquet(data_path / "session_settings.parquet", engine="pyarrow")
session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)

print(f"✅ Finished loading all parquet files")




In [ ]:
#---------------------------------------------------
# PLOT FIGURE to ascertain everything is well loaded
#---------------------------------------------------

df_to_analyze = photometry_tracking_encoder_data["Photodiode_int"] #using downsampled values in common time grid 
#df_to_analyze = camera_photodiode_data["Photodiode"] #use async raw values if needed for troubleshooting, but the nearest indices needs to be found , see couple of lines below
photodiode_halts, photodiode_delay_min, photodiode_delay_avg, photodiode_delay_max = process.analyze_photodiode(df_to_analyze, experiment_events, event_name, plot = True)
# nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
# photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices]
process.plot_figure_1(photometry_tracking_encoder_data, session_name, save_path, common_resampled_rate, photodiode_halts, save_figure = False, show_figure = True, downsample_factor=50)

In [ ]:
# Force reload the modules
importlib.reload(harp_resources.process)
importlib.reload(harp_resources.utils)
# Reassign after reloading to ensure updated references
process = harp_resources.process
utils = harp_resources.utils

In [ ]:
#-----------------------------------------
# Get Block times and durations
#-----------------------------------------

block_events = experiment_events[
    experiment_events["Event"].str.contains("block started|Block timer elapsed", case=False, na=False)
].copy()  # Use .copy() to avoid SettingWithCopyWarning
block_events["Time Difference"] = block_events.index.to_series().diff().dt.total_seconds()
block_events = block_events.drop(columns=['Seconds'])

# Print results
print(block_events)


In [ ]:
# Extract times when the event "Applying halt: 2s" occurs
halt_times = experiment_events[experiment_events["Event"].str.contains("Apply halt: 2s")].index
print(halt_times)

# Create an empty DataFrame to store aligned data
aligned_data = []

# Loop through each halt event time
for halt_time in halt_times:
    # Extract data in the [-1s, +5s] window
    window_data = photometry_tracking_encoder_data.loc[
        (photometry_tracking_encoder_data.index >= halt_time - pd.Timedelta(seconds=1)) &
        (photometry_tracking_encoder_data.index <= halt_time + pd.Timedelta(seconds=5))
    ].copy()

    # Compute time relative to halt
    window_data["Time (s)"] = (window_data.index - halt_time).total_seconds()
    
    # Add event identifier
    window_data["Halt Time"] = halt_time

    # Store aligned data
    aligned_data.append(window_data)

# Concatenate all windows
aligned_df = pd.concat(aligned_data)

# Plot the data
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot Photodiode_int on the primary y-axis
for halt_time in halt_times:
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    ax1.plot(subset["Time (s)"], subset["Photodiode_int"], label=f'Photodiode at {halt_time}')

ax1.set_xlabel('Time (s) relative to halt')
ax1.set_ylabel('Photodiode_int')
ax1.set_title('Photodiode and z_470 Signals Triggered by "Applying halt: 2s" Events')

# Create a secondary y-axis for z_470
ax2 = ax1.twinx()
for halt_time in halt_times:
    subset = aligned_df[aligned_df["Halt Time"] == halt_time]
    ax2.plot(subset["Time (s)"], subset["z_470"], label=f'z_470 at {halt_time}', linestyle='--')

ax2.set_ylabel('z_470')

# Show the plot
fig.tight_layout()
plt.show()

NEXT - get event triggered averages of all relevant data and reproduce figure below 
**THIS IS NOT WORKING, SUGGEST 1. downsample photodiode signal and well and save togerther with dataset.**

NEXT - think about / run some correlation analysis 

In [ ]:
halt_times

In [ ]:
pympler_memory_df = utils.get_pympler_memory_usage(top_n=10)
